### Load the required libraries

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
#from sklearn.metrics import ConfusionMatrixDisplay

from numpy import asarray
from numpy import savetxt

In [3]:
import sklearn
sklearn.__version__

'1.0.2'

# Spruce training

In [5]:
input_file_wood = pd.read_csv('Spruce1_wood_training_in.txt',sep=' ') # input wood training data 
input_file_leaf = pd.read_csv('Spruce1__leaf_training_in.txt',sep=' ') # input leaf training data
# input_file_lichen = pd.read_csv('spruce_training_lichen.txt', sep=',')

# input_file_wood["label"] = 1 # defines labels for the wood training data
# input_file_leaf["label"] = 0 # defines labels for the leaf training data # don't need this because I classified it using the tool in CC, but useful for future probably
#input_file_lichen["label"] = 2

# input_file = pd.concat([input_file_wood,input_file_leaf]) # create one file to perform classification

# Larch training 


In [7]:
input_file_wood = pd.read_csv('larch_training_wood.txt',sep=',') # input wood training data 
input_file_leaf = pd.read_csv('larch_training_leaf.txt',sep=',') # input leaf training data
# input_file_lichen = pd.read_csv('larch_training_lichen.txt', sep=',') # input lichen training data, all data separated by comma
input_file = pd.concat([input_file_wood, input_file_leaf]) # create one file to perform classification

In [8]:
#input_file.shape, input_file_wood.shape[0]+input_file_leaf.shape[0]+input_file_lichen.shape[0] # defines shape of array, makes sure the collumns are the same size (116860), example output: ((9391, 11), 9391) 
input_file.shape, input_file_wood.shape[0]+input_file_leaf.shape[0] # defines shape of array, makes sure the columns are the same size (116860), example output: ((9391, 11), 9391) 

((9293, 11), 9293)

In [9]:
# input_file # view concatenated file

input_file_clean = input_file.drop(input_file.index[[0]]) # need for larch, I had NANs in first row, might not need this in the future
input_file_clean # view concatenated file

,//X,Y,Z,R,G,B,NumberOfReturns,ReturnNumber,GpsTime,Intensity,Classification
1,-6.81500,-8.17975,-1.76525,121.0,136.0,99.0,1.0,1.0,144.594065,50964.0,1.0
2,-6.84100,-8.16275,-1.73825,74.0,89.0,40.0,1.0,1.0,144.635158,48452.0,1.0
3,-6.87000,-8.12800,-1.44300,155.0,138.0,54.0,2.0,2.0,144.696739,43711.0,1.0
4,-6.83300,-8.16825,-1.73350,122.0,127.0,39.0,2.0,1.0,138.936783,44105.0,1.0
5,-6.80350,-8.18225,-1.77650,168.0,166.0,83.0,1.0,1.0,138.854603,46901.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
4470,-6.93900,-7.73675,-0.83200,164.0,167.0,80.0,2.0,2.0,262.808039,23112.0,0.0
4471,-6.94350,-7.72750,-0.83350,161.0,166.0,75.0,2.0,2.0,262.787500,22893.0,0.0
4472,-6.98300,-7.69300,-0.80700,166.0,156.0,70.0,3.0,2.0,262.705286,28048.0,0.0
4473,-6.69775,-8.10275,-1.56150,151.0,163.0,81.0,1.0,1.0,272.796900,19704.0,0.0


In [11]:
# x = np.array(input_file[["Intensity", "NumberOfReturns"]]) # spruce, defines what parameters will be used to train the model, validating the model
# y = np.array(input_file[['label']]) # use for spruce
x = np.array(input_file_clean[["Intensity", "NumberOfReturns"]]) # defines what parameters will be used to train the model, validating the model, # use for larch
y = np.array(input_file_clean[["Classification"]]) # labels the point as leaf, wood, or lichen (use this for larch)


In [12]:
accuracy_list = list()#[] # define empty list

In [13]:
kf = KFold(n_splits = 10, shuffle = True) # define how many sections to partition the data into, shuffle the training data 
for train, val in kf.split(x):
        x_train, x_val, y_train, y_val = x[train], x[val], y[train].ravel(), y[val].ravel() # define the subsets for the partitions 
        clf = RandomForestClassifier(random_state=0,class_weight="balanced_subsample") # parameters for random forest, depth --> how many nodes in the tree, random state = 0 --> same input file, same result
        clf.fit(x_train, y_train) # fit model to training data
        y_val_predict = clf.predict(x_val) # using intensity (reflectance) and number of returns values to predict leaf, wood, or lichen
        
        # take validation and measure the performance of the random forest that we trained 
        acc_score = accuracy_score(y_val, y_val_predict)
        accuracy_list.append(acc_score) # store accuracy values in accuracy list, 10 values because 10 cross fold
        
        # compute the average overall accuracy 
        mean_accuracy = np.mean(accuracy_list)
        
        print(f"Mean Accuracy: {mean_accuracy}")


Mean Accuracy: 0.9967741935483871
Mean Accuracy: 0.9962342442445918
Mean Accuracy: 0.9967718786531939
Mean Accuracy: 0.9975789089898954
Mean Accuracy: 0.9976325566859959
Mean Accuracy: 0.9976683218167297
Mean Accuracy: 0.9973863179773107
Mean Accuracy: 0.9975784749470468
Mean Accuracy: 0.9974887245313302
Mean Accuracy: 0.9974169241987569


# Testing trained model on all current trees

In [14]:
clf = RandomForestClassifier(random_state=0) # parameters for random forest, depth --> how many nodes in the tree, random state = 0 --> same input file, same result
clf.fit(x, y.ravel()) # use the whole data set

y_train_predict = clf.predict(x)
acc_score = accuracy_score(y, y_train_predict)
print(f"Train Accuracy: {acc_score}")

Train Accuracy: 0.9984931654289096


# Testing on tree model has not seen

In [16]:
input_file_test = pd.read_csv('C:\\Users\\ANGELASEIBERT\\Desktop\\Python_Workspace\\plot_footprint\\plot4_2017_trees_fullfp.txt', sep=',') # load unclassified tree

In [17]:
input_file_test # check the columns of the input file if needed

,//X,Y,Z,EdgeOfFlightLine,NumberOfReturns,ReturnNumber,Intensity,Coord. Z
0,173445.555711,83757.336865,412.893806,0.0,2.0,2.0,14068.0,NaN
1,173449.041832,83757.209825,412.682185,0.0,1.0,1.0,39626.0,NaN
2,173449.024238,83757.209734,412.728628,0.0,1.0,1.0,44432.0,NaN
3,173449.025126,83757.208550,412.733391,0.0,1.0,1.0,43384.0,NaN
4,173449.031402,83757.206144,412.734181,0.0,1.0,1.0,37267.0,NaN
...,...,...,...,...,...,...,...,...
5456495,173448.192224,83750.115235,417.956884,0.0,1.0,1.0,14155.0,6.356884
5456496,173448.198612,83750.114805,417.953158,0.0,1.0,1.0,28726.0,6.353158
5456497,173448.195940,83750.112893,417.954164,0.0,1.0,1.0,18153.0,6.354164
5456498,173448.191951,83750.120732,417.959338,0.0,1.0,1.0,20883.0,6.359338


# Generate the classified file

In [18]:
# Testing
input_file_test = input_file_test.drop(['Coord. Z'], axis = 1) # CHANGE IF NEEDED: if working with separate species, drop coord z and origninal cloud index (oci comes up if you merge clouds)
x_test = np.array(input_file_test[["Intensity","NumberOfReturns"]]) # define what columns of data to use
#y_test = np.array(input_file_test[["label"]]) # don't need unless you are actually testing and want an accuracy value. this is a labeled test set to compare to what the model puts out

y_test_predict = clf.predict(x_test) # classify the tree
y_test_predict # show array of predictions

array([0., 0., 1., ..., 0., 0., 0.])

# concatenate files 

In [20]:
# exporting and importing the point clouds over the past few years has led to inconsisitencies, so we need to manually match them up. 


y_test_predict_df = pd.DataFrame(y_test_predict)
input_file_test_df = pd.DataFrame(input_file_test)

concat_dfs = pd.concat([input_file_test_df, y_test_predict_df], axis = 1)
concat_dfs.columns = ["//X", "Y", "Z", "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "Intensity", "Classification"]
# concat_dfs.columns = ["//X", "Y", "Z","R", "G", "B", "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", "Classification"]
# concat_dfs.columns = ["//X", "Y", "Z", 'Coord. Z', "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", "Classification"]
# concat_dfs.columns = ["//X", "Y", "Z","R", "G", "B", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", "Classification"]
# concat_dfs.columns = ["//X", "Y", "Z", "R", "G", "B", 'Amplitude', "Reflectance", "Deviation",  "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", "Classification"] # removing rgb makes the plot visualize as elevation in open3d
# concat_dfs.columns = ["//X", "Y", "Z", "R", "G", "B", "Reflectance", "Deviation","EdgeOfFlightLine", "GpsTime", "Intensity", "Classification"]
# concat_dfs.columns = ["//X", "Y", "Z", "Original cloud index", "Deviation", "Reflectance", "Amplitude", "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", "Classification"]
# concat_dfs.columns = ["//X", "Y", "Z", "R", "G", "B", 'Coord. Z', "Deviation","Reflectance", 'Amplitude', "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", "Classification"] # removing rgb makes the plot visualize as elevation in open3d
# concat_dfs.columns = ["//X", "Y", "Z", "Amplitude", "Reflectance", "Deviation", "EdgeOfFlightLine", "NumberOfReturns", "ReturnNumber", "GpsTime", "Intensity", 'Coord. Z', "Classification"]

concat_dfs


,//X,Y,Z,EdgeOfFlightLine,NumberOfReturns,ReturnNumber,Intensity,Classification
0,173445.555711,83757.336865,412.893806,0.0,2.0,2.0,14068.0,0.0
1,173449.041832,83757.209825,412.682185,0.0,1.0,1.0,39626.0,0.0
2,173449.024238,83757.209734,412.728628,0.0,1.0,1.0,44432.0,1.0
3,173449.025126,83757.208550,412.733391,0.0,1.0,1.0,43384.0,1.0
4,173449.031402,83757.206144,412.734181,0.0,1.0,1.0,37267.0,0.0
...,...,...,...,...,...,...,...,...
5456495,173448.192224,83750.115235,417.956884,0.0,1.0,1.0,14155.0,0.0
5456496,173448.198612,83750.114805,417.953158,0.0,1.0,1.0,28726.0,0.0
5456497,173448.195940,83750.112893,417.954164,0.0,1.0,1.0,18153.0,0.0
5456498,173448.191951,83750.120732,417.959338,0.0,1.0,1.0,20883.0,0.0


# Save concatenated df 

In [21]:
concat_dfs.to_csv('C:\\Users\\ANGELASEIBERT\\Desktop\\Python_Workspace\\plot_footprint\\plot4_2014_classifiedfp.csv', index = False) # saves classified file without numbering rows